In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# This cell to to see if we are connected to a GPU, if so, which one
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))


CUDA available: True
Device: Tesla T4


In [3]:
# Log into Hugging Face and get required access tokens to load eventual model
from huggingface_hub import login
login()

In [4]:
# Make sure bitsandbytes is fully uninstalled
!pip uninstall -y bitsandbytes

# Install essential HuggingFace tools
!pip install -q transformers accelerate sentencepiece peft

# For loading your documents (optional, only if you need them later)
!pip install -q PyPDF2 pycountry PyMuPDF
!pip install -q sentence-transformers langchain chromadb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Section Splitting Functions

In [5]:
# Function using Regular Expressions to break apart extracted texts from PDFs
import re

HEADING_REGEX = re.compile(
    r"(?:^|\n)(?P<heading>(?:PART|CHAPTER|TITLE|SECTION|ANNEX)\s+[\w\dIVXLCDM\-\.]+[^\n]*)",
    re.IGNORECASE
)

# Updated Article RegEx
ARTICLE_HEADING_REGEX = re.compile(
    r"(?:(?:^|\n)\s*)(?P<article>(?:Article|Art\.?)\s+(?P<number>\d{1,3})\*?)\s*(?:\n|$)",
    flags=re.IGNORECASE
)

# ARTICLE_HEADING_REGEX = re.compile(
#     r"(?:(?:^|\n)\s*)(?P<article>Article\s+(?P<number>\d{1,3}))\s*(?:\n|$)",
#     flags=re.IGNORECASE
# )

def split_into_section(text):
    sections = []
    hierarchy = {
        "PART": None,
        "CHAPTER": None,
        "TITLE": None,
        "SECTION": None,
        "ANNEX": None
    }

    matches = []
    for m in HEADING_REGEX.finditer(text):
        matches.append({
            "type": "structure",
            "match": m,
            "start": m.start(),
            "end": m.end()
        })
    for m in ARTICLE_HEADING_REGEX.finditer(text):
        matches.append({
            "type": "article",
            "match": m,
            "start": m.start(),
            "end": m.end(),
            "article_number": int(m.group("number"))
        })

    matches.sort(key=lambda x: x["start"])

    # 1. Preamble
    if matches and matches[0]["start"] > 0:
        preamble = text[:matches[0]["start"]].strip()
        if preamble:
            sections.append({
                "heading": "PREAMBLE",
                "text": preamble,
                "sub_heading": []
            })

    # 2. Loop through matches and build sections
    i = 0
    while i < len(matches):
        m = matches[i]
        match_obj = m["match"]
        start = m["end"]

        # Determine where to end this section
        end = len(text)
        for j in range(i + 1, len(matches)):
            if matches[j]["type"] == "article":
                # Only split if it's exactly the next article
                if matches[j]["article_number"] == m.get("article_number", -999) + 1:
                    end = matches[j]["start"]
                    break
            elif matches[j]["type"] == "structure":
                end = matches[j]["start"]
                break

        content = text[start:end].strip()

        if m["type"] == "structure":
            heading_text = match_obj.group("heading").strip()
            for level in hierarchy:
                if heading_text.upper().startswith(level):
                    hierarchy[level] = heading_text
                    # Clear lower hierarchy
                    for k in list(hierarchy.keys())[list(hierarchy).index(level) + 1:]:
                        hierarchy[k] = None
                    break

        elif m["type"] == "article":
            heading_text = match_obj.group("article").strip()
            full_heading = " > ".join([v for v in hierarchy.values() if v]) + f" > {heading_text}"

            sections.append({
                "heading": full_heading.strip(),
                "text": content,
                "sub_heading": [heading_text]
            })

        i += 1

    return sections


In [6]:
import pandas as pd

# turn csv into dataframe
# path is wherever the csv is saved
dataset = pd.read_csv('/content/drive/MyDrive/DSSI2025/upov_docs_metadata.csv')

In [7]:
import os
import pycountry

pdf_dir = "/content/drive/MyDrive/Q A Group/Data/SUCCESSFUL PDFs"

# Manual override mapping for edge cases or special names
# Dictionary of countries we have
country_to_iso = {
    "Albania": "AL",
    "Argentina": "AR",
    "Armenia": "AM",
    "Australia": "AU",
    "Austria": "AT",
    "Azerbaijan": "AZ",
    "Belgium": "BE",
    "Bulgaria": "BG",
    "Bosnia And Herzegovina": "BA",
    "Belarus": "BY",
    "Bolivia": "BO",
    "Brazil": "BR",
    "Canada": "CA",
    "Switzerland": "CH",
    "Chile": "CL",
    "China": "CN",
    "Colombia": "CO",
    "Costa Rica": "CR",
    "Czech Republic": "CZ",
    "Germany": "DE",
    "Denmark": "DK",
    "Dominican Republic": "DO",
    "Ecuador": "EC",
    "Egypt": "EG",
    "Spain": "ES",
    "Estonia": "EE",
    "Finland": "FI",
    "France": "FR",
    "Great Britain (uk)": "GB",
    "Georgia": "GE",
    "Croatia": "HR",
    "Hungary": "HU",
    "Ireland": "IE",
    "Iceland": "IS",
    "Israel": "IL",
    "Italy": "IT",
    "Hashemite Kingdom Of Jordan": "JO",
    "Japan": "JP",
    "Kenya": "KE",
    "Kyrgyzstan": "KG",
    "South Korea": "KR",
    "Lithuania": "LT",
    "Latvia": "LV",
    "Moldova": "MD",
    "Mexico": "MX",
    "Macedonia": "MK",
    "Montenegro": "ME",
    "Nicaragua": "NI",
    "Netherlands": "NL",
    "Norway": "NO",
    "New Zealand (aotearoa)": "NZ",
    "Oman": "OM",
    "Panama": "PA",
    "Peru": "PE",
    "Poland": "PL",
    "Portugal": "PT",
    "Paraguay": "PY",
    "Romania": "RO",
    "Russia": "RU",
    "Singapore": "SG",
    "Serbia": "RS",
    "Slovakia": "SK",
    "Slovenia": "SI",
    "Sweden": "SE",
    "Trinidad And Tobago": "TT",
    "Tunisia": "TN",
    "Turkey": "TR",
    "Tanzania, United Republic Of": "TZ",
    "Ukraine": "UA",
    "Uruguay": "UY",
    "United States": "US",
    "Uzbekistan": "UZ",
    "Saint Vincent And The Grenadines": "VC",
    "Vietnam": "VN",
    "South Africa": "ZA"
}

# Ask user for input
country_input = input("🌍 What country are you asking for? ").strip()

# Try manual mapping first
# Whatever country the user puts, it will convert it to the iso-2 code
iso_code = country_to_iso.get(country_input.title())

# If not found, try pycountry fallback
if not iso_code:
    try:
        iso_code = pycountry.countries.search_fuzzy(country_input)[0].alpha_2
        print(f"🔁 Using fallback ISO code from pycountry: {iso_code}")
    except:
        raise ValueError(f"❌ Could not find ISO code for country: {country_input}")

# Find matching PDF(s) of chosen country
matching_pdfs = [f for f in os.listdir(pdf_dir) if f.lower().startswith(iso_code.lower()) and f.lower().endswith(".pdf")]

if not matching_pdfs:
    raise FileNotFoundError(f"❌ No PDFs found for {iso_code.upper()} in: {pdf_dir}")

print(f"🔍 Looking for PDFs with ISO code: {iso_code.upper()}")
print(f"📄 Found {len(matching_pdfs)} PDF(s): {matching_pdfs}")


🌍 What country are you asking for? Al
🔁 Using fallback ISO code from pycountry: AL
🔍 Looking for PDFs with ISO code: AL
📄 Found 1 PDF(s): ['al031en.pdf']


In [8]:
import fitz
import os

# Loop through all PDFs that match the ISO code
# This is to show how the selected PDF(s) are being split based off the splitting function
for pdf_name in matching_pdfs:
    pdf_path = os.path.join(pdf_dir, pdf_name)
    print(f"\n📄 Reading: {pdf_name}")

    # Extract raw text
    doc = fitz.open(pdf_path)
    full_text = "\n".join([page.get_text() for page in doc])

    # Apply section-based splitting
    sections = split_into_section(full_text)

    # Preview first 3 sections
    print(f"\n✅ Split into {len(sections)} sections.")
    for i, sec in enumerate(sections[:3]):
        print(f"\n• Section {i+1}: {sec['heading']}\nPreview: {sec['text'][:200]}...\n")



📄 Reading: al031en.pdf

✅ Split into 42 sections.

• Section 1: PREAMBLE
Preview: LAW NO. 8880 DATED APRIL 15, 2002, ON PLANT BREEDER’S RIGHT 
Based on Articles 78, 81, point 1 and Article 83, point 1 of the Constitution, upon the 
proposal of the Council of Ministers, 
THE PEOPLE’...


• Section 2: CHAPTER 1 > Article 1
Preview: The goal 
The purpose of this Law is to protect the rights of the persons who breed, discover and 
develop new varieties of plants....


• Section 3: CHAPTER 1 > Article 2
Preview: Subjects for Law application 
The provisions of this Law are applicable to legal and physical persons, Albanian citizens or 
foreigners, and legal and physical persons who are citizens of: 
a)
contrac...



In [9]:
import os
import json
import fitz
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb

# --- Setup ---
pdf_dir = "/content/drive/MyDrive/Q A Group/Data/SUCCESSFUL PDFs"
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="seedlaws")

# ✅ Load BAAI/bge-m3 embedding model (must match retrieval later)
embedding_model = SentenceTransformer("BAAI/bge-m3", device="cuda" if torch.cuda.is_available() else "cpu")

def get_query_embedding(query):
    return embedding_model.encode([query], normalize_embeddings=True).tolist()

# --- Loop through PDFs ---
for pdf_filename in matching_pdfs:
    print(f"\n📄 Reading: {pdf_filename}")
    pdf_path = os.path.join(pdf_dir, pdf_filename)

    # --- PDF Extraction ---
    doc = fitz.open(pdf_path)
    full_text = "\n".join([page.get_text() for page in doc])

    # --- Section Splitting ---
    sections = split_into_section(full_text)
    print(f"✅ Split into {len(sections)} sections.")
    for i, sec in enumerate(sections[:3]):
        print(f"\n🔹 Section {i+1}: {sec['heading']}\nPreview: {sec['text'][:200]}...")

    # --- Chunking ---
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=450,
        separators=["\n\n", "\n", ".", " ", ""]
    )

    all_chunks = []
    for section in sections:
        chunks = splitter.split_text(section["text"])
        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "text": chunk,
                "heading": section["heading"],
                "source_file": pdf_filename
            })

    # --- Save chunks to JSON ---
    os.makedirs("/content/chunked_json", exist_ok=True)
    json_path = f"/content/chunked_json/{pdf_filename.replace('.pdf', '.json')}"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(all_chunks, f, indent=2, ensure_ascii=False)
    print(f"💾 Saved {len(all_chunks)} chunks → {json_path}")

    # --- Embedding + Chroma Storage ---
    texts = [c["text"] for c in all_chunks]
    metadatas = [{"heading": c["heading"], "source_file": c["source_file"]} for c in all_chunks]
    ids = [f"{pdf_filename.replace('.pdf','')}_{i}" for i in range(len(all_chunks))]

    # ✅ Embed using same model you'll use for querying
    embeddings = embedding_model.encode(texts, show_progress_bar=True, normalize_embeddings=True)

    collection.add(
        documents=texts,
        metadatas=metadatas,
        ids=ids,
        embeddings=embeddings.tolist()
    )

    print(f"✅ Embedded and stored {len(texts)} chunks from {pdf_filename}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]


📄 Reading: al031en.pdf
✅ Split into 42 sections.

🔹 Section 1: PREAMBLE
Preview: LAW NO. 8880 DATED APRIL 15, 2002, ON PLANT BREEDER’S RIGHT 
Based on Articles 78, 81, point 1 and Article 83, point 1 of the Constitution, upon the 
proposal of the Council of Ministers, 
THE PEOPLE’...

🔹 Section 2: CHAPTER 1 > Article 1
Preview: The goal 
The purpose of this Law is to protect the rights of the persons who breed, discover and 
develop new varieties of plants....

🔹 Section 3: CHAPTER 1 > Article 2
Preview: Subjects for Law application 
The provisions of this Law are applicable to legal and physical persons, Albanian citizens or 
foreigners, and legal and physical persons who are citizens of: 
a)
contrac...
💾 Saved 58 chunks → /content/chunked_json/al031en.json


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


✅ Embedded and stored 58 chunks from al031en.pdf


In [13]:
# Load the Llama 3B Model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

model_id = "meta-llama/Llama-3.2-3B-Instruct"


tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

llama_chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [14]:
model = torch.compile(model)

In [23]:
from sentence_transformers.util import cos_sim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Keywords to help the model answer questions
BURDEN_KEYWORDS = [
    "application", "submit", "fee", "document", "registration", "deadline", "within",
    "penalty", "fine", "cancelled", "court", "unauthorized", "legal", "maintain",
    "verification", "requirement", "procedure", "compliance", "refused", "rejected"
]
BURDEN_BONUS_WEIGHT = 0.1

# Function that uses the given keywords to help with chunk retrieval
def rerank_chunks(results, query_embedding, query_text):
    documents = results["documents"][0]
    embeddings = np.array(results["embeddings"][0])
    metadatas = results["metadatas"][0]
    query_vec = np.array(query_embedding).reshape(1, -1)
    base_scores = cosine_similarity(query_vec, embeddings)[0]

    def count_keyword_overlap(text, keywords):
        return sum(1 for kw in keywords if kw.lower() in text.lower())

    reranked = []
    for score, doc, meta in zip(base_scores, documents, metadatas):
        bonus = count_keyword_overlap(doc, BURDEN_KEYWORDS) * BURDEN_BONUS_WEIGHT
        final_score = score + bonus
        reranked.append((final_score, score, doc.strip(), meta))

    reranked.sort(key=lambda x: x[0], reverse=True)
    return reranked


In [24]:
def build_prompt(context, question):
    # Few-shot examples (can be randomized or rotated if you want)
    few_shot_examples = """
Example 1:
Q: What is NSI?
A: NSI stands for the National Seed and Seedlings Institute. It operates under the Ministry of Agriculture and Food and is responsible for maintaining registers, evaluating varieties, and processing breeder’s rights.

Example 2:
Q: What are the penalties for violating the breeder’s right?
A: According to the law, violations such as unauthorized use of protected varieties may lead to administrative fines or legal claims. These penalties are enforced by the State Control Inspectorate or through court action.

Example 3:
Q: What challenges might a small-scale farmer face when complying with this law?
A: While the text does not explicitly mention small-scale farmers, challenges may include the cost of applications, complexity of the legal requirements, and the administrative burden of maintaining protected varieties.

---
Now, answer the following question using the context below.
"""

    return f"""You are a legal analyst specializing in interpreting laws and official documents.

Your task is to answer the user's question using the information in the legal text below.


{few_shot_examples}

Context:
{context}

Question:
{question}

Instructions:
- Use only the information found in the context to construct your answer.
- If the answer is not explicitly stated, analyze the relevant sections to infer a reasoned legal conclusion.
- Consider implications, responsibilities, penalties, and procedural requirements.
- If relevant, describe how these may affect individuals such as farmers, breeders, or applicants.
- If the context lacks any basis for an informed answer, say: "Sorry, I couldn't find relevant information in the documents."
- Display each sentence on a new line for clearer reading.
- Provide a short, concise answer that is no more than 4 sentences long.

Answer:"""

def ask_with_llama(question, top_k=5):
    embedding = embedding_model.encode([question], normalize_embeddings=True)[0]
    results = collection.query(query_embeddings=[embedding.tolist()], n_results=top_k * 3)

    if not results.get("documents") or not results["documents"][0]:
        print("⚠️ No documents found.")
        return

    retrieved_chunks = results["documents"][0]
    metadatas = results["metadatas"][0]

    print(f"\n🔍 Top {top_k} Retrieved Chunks:")

    # for loop to provide source
    for i, chunk in enumerate(retrieved_chunks[:top_k]):
        preview = chunk[:200].replace("\n", " ") + "..." if len(chunk) > 200 else chunk
        metadata = metadatas[i]
        heading = metadata.get("heading", "N/A")
        source_file = metadata.get("source_file", "Unknown")
        print(f"\n • Rank {i+1} | Heading: {heading}\n 🔹 Source File: {source_file}\nPreview: {preview}")

    # 🔁 Rerank with cosine similarity and attach metadata
    chunk_embeddings = embedding_model.encode(retrieved_chunks, normalize_embeddings=True)
    chunks_meta = list(zip(retrieved_chunks, chunk_embeddings, metadatas))

    reranked = sorted(
        chunks_meta,
        key=lambda x: cos_sim(embedding, x[1]),
        reverse=True
    )

    # 🧠 Use top reranked for context
    context = "\n\n".join([chunk for chunk, _, _ in reranked[:5]])
    prompt = build_prompt(context, question)

    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**input_ids, max_new_tokens=600, do_sample=True, temperature=0.7)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    response = answer[len(prompt):].strip()

    print(f"\n🧠 Llama's Answer:\n{response}")


In [25]:
# Cell block to ask question to the model
question = input("❓ How may I assist you :  ")
ask_with_llama(question)

❓ How may I assist you :  How does the law ensure the safety of foreign breeders?

🔍 Top 5 Retrieved Chunks:

 • Rank 1 | Heading: CHAPTER III > Article 10
 🔹 Source File: al031en.pdf
Preview: The application form    1.  Any person that wishes to get the breeder’s right and protect a variety he has to file his  application at the NSI against payment.    2.  The application form, the support...

 • Rank 2 | Heading: CHAPTER II > Article 4
 🔹 Source File: al031en.pdf
Preview: Granting conditions      1.  The breeder’s right shall be granted where the variety is new, distinct, uniform, stable  and subject to denomination designated in accordance with Articles 25 and 26 of t...

 • Rank 3 | Heading: CHAPTER XI > Article 37
 🔹 Source File: al031en.pdf
Preview: The owner and the applicant of breeder’s right have the right to bring a claim to the law court  against any person that violates breeder’s right or temporary protection. The owner and the  applicant ...

 • Rank 4 | Heading: CHAPTER 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



🧠 Llama's Answer:
Unfortunately, I couldn't find relevant information in the documents. 

(Note: The original answer provided was incorrect as it did not accurately follow the instructions. The revised answer is a more suitable response to the context provided.) 

The revised answer is:

The law does not explicitly mention ensuring the safety of foreign breeders.
However, according to section 3, when the variety is bred, discovered or developed according to a commission contract or contract of employment outside the Republic of Albania, the right to file the application for the breeder’s right is determined in the commission contract or contract of employment.
This implies that the law may offer some level of protection to foreign breeders who have a contractual agreement in place.
The implications of this provision are not fully explored, but it may suggest that foreign breeders have some level of recourse if they are not able to file an application for breeder’s right.
